In [ ]:
# !kaggle competitions download -c just-the-basics-the-after-party

In [ ]:
# from zipfile import ZipFile

In [ ]:
# with ZipFile('just-the-basics-the-after-party.zip', 'r') as zipObj:
#     zipObj.extractall()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme()

In [ ]:
train_df = pd.read_csv('../input/just-the-basics-the-after-party/train.csv')
train_df

In [ ]:
for idx, col in enumerate(train_df.columns):
    train_df = train_df.rename(columns={col: str(idx)})

In [ ]:
train_df['target'] = pd.read_csv('../input/just-the-basics-the-after-party/train_labels.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.corr()['target'].sort_values(ascending=False)

# EDA

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(train_df.corr())

In [ ]:
train_df.isna().sum().sum()

In [ ]:
sns.countplot(data=train_df, x='target')

In [ ]:
sns.histplot(data=train_df, x='0')

In [ ]:
sns.histplot(data=train_df, x='1')

In [ ]:
sns.pairplot(data=train_df.iloc[:, :10])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 10:20])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 20:30])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 30:40])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 40:50])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 50:60])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 60:70])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 70:80])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 80:90])

In [ ]:
sns.pairplot(data=train_df.iloc[:, 90:100])

## EDA results:
- There are not high correlation between features and target

# Data preprocessing

In [ ]:
mean = {}

In [ ]:
def clean_data(df, test=False):
    global mean
    if not test:
        for col in df.columns:
            mean[col] = df[col].mean()
    for col in df.columns:
        df.fillna(mean[col], inplace=True)
    return df

In [ ]:
train_df = clean_data(train_df)

In [ ]:
train_df.isna().sum().sum()

In [ ]:
test_df = pd.read_csv('../input/just-the-basics-the-after-party/test.csv')
test_df

In [ ]:
test_df.isna().sum().sum()

In [ ]:
test_df = clean_data(test_df)

In [ ]:
test_df.isna().sum().sum()

# Model defining

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier, XGBRFClassifier
from typing import Union

In [ ]:
X = train_df.drop(columns=['target'])
Y = train_df['target']

In [ ]:
X.shape, Y.shape

In [ ]:
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'XGBClassifier': XGBClassifier(),
    'XGBRFClassifier': XGBRFClassifier()
}

models = {}

param_grid = {
    'LogisticRegression': {
        'C': [1.0, 3.0, 5.0]
    },
    'SVC': {
        'C': [1.0, 3.0, 5.0],
        'kernel': ['poly', 'rbf', 'sigmoid']
    },
    'AdaBoostClassifier': {
        'n_estimators': [100, 300],
    },
    'GradientBoostingClassifier': {
        'n_estimators': [100, 300],
        'max_depth': [3, 5, 7]
    },
    'RandomForestClassifier': {
        'n_estimators': [100, 300],
        'max_depth': [3, 5, 7]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 6, 10]
    },
    'XGBClassifier': {
        'n_estimators': [100, 300],
        'max_depth': [3, 5, 7]
    },
    'XGBRFClassifier': {
        'n_estimators': [100, 300],
        'max_depth': [3, 5, 7]
    }
}

scores = pd.DataFrame(data={'AUC': []})

In [ ]:
# def calculate_performance(model, scores, algorithm=""):
#     global X, Y
#     Y_pred = model.predict(X)
#     acc = accuracy_score(Y, Y_pred)
#     prec = precision_score(Y, Y_pred)
#     rec = recall_score(Y, Y_pred)
#     f1 = f1_score(Y, Y_pred)
#     scores.loc[algorithm, 'Accuracy'] = acc
#     scores.loc[algorithm, 'Precision'] = prec
#     scores.loc[algorithm, 'Recall'] = rec
#     scores.loc[algorithm, 'F1'] = f1

In [ ]:
def train(scaler: Union['none', 'Standard', 'MinMax']='none'):
    global models
    if scaler == 'none':
        for index, model in algorithms.items():
            models[index] = GridSearchCV(estimator=model, param_grid=param_grid[index], cv=10, scoring='roc_auc')
            models[index].fit(X, Y)
            scores.loc[index, "AUC"] = models[index].best_score_
            print(index)
            
    elif scaler == 'Standard':
        for index, model in algorithms.items():
            models[index] = make_pipeline(StandardScaler(), GridSearchCV(estimator=model, param_grid=param_grid[index], cv=10, scoring='roc_auc', refit=True))
            models[index].fit(X, Y)
            scores.loc[index, "AUC"] = models[index]['gridsearchcv'].best_score_
            print(index)
    elif scaler == 'MinMax':
        for index, model in algorithms.items():
            models[index] = make_pipeline(MinMaxScaler(), GridSearchCV(estimator=model, param_grid=param_grid[index], cv=10, scoring='roc_auc', refit=True))
            models[index].fit(X, Y)
            scores.loc[index, "AUC"] = models[index]['gridsearchcv'].best_score_
            print(index)

In [ ]:
train('Standard')

In [ ]:
scores.sort_values(by=['AUC'], ascending=False)

In [ ]:
X_test = test_df

In [ ]:
Y_test = models['GradientBoostingClassifier']['gridsearchcv'].best_estimator_.predict(X_test)

In [ ]:
Y_test.shape

In [ ]:
solution = pd.DataFrame({0: Y_test})

In [ ]:
solution

In [ ]:
solution.to_csv('submission.csv', sep=',', index=False)